# СЕРВИС ФОРМИРОВАНИЯ ЗАДАЧ ДЛЯ МОСКВИЧЕЙ ПО КОНТРОЛЮ РАБОТ ПОДРЯДЧИКОВ В СФЕРЕ ГОРОДСКОГО БЛАГОУСТРОЙСТВА
## Цель
**Разработка сервиса, который с помощью распознавания документации проектов городского благоустройства сформирует для москвичей задания по мониторингу выполненных подрядчиками работ**

## 1. Открываем смету
### 1.1. Необходимо создать функцию:
- открываем смету,
- вытаскиваем адрес,
- наименование или индекс листа - задает пользователь,
- обрабатываем от пропусков и шапок (вверху, внизу),
- переименовываем столбцы, чтобы совпадали названия в справочниках СПГЗ и КПГЗ:
    - интеграция со справочником СН и ТСН,
    - сравниваем со справочником СПГЗ и КПГЗ;
- обработка результата (см. ниже)- вывести в отдельный файл. 

## 2. Разделы и работы, в которых употребляются словосочетания ниже, не отражающие суть ключевых работ являются исключениями и не должны участвовать в дальнейшем разборе сметы: 
- Вывоз мусора 
- Погрузка мусора 
- Перевозка мусора 
- Возвратные материалы 
- Пуско-наладочные работы и т.д. 

Данные исключения сформированы в сервисе в виде справочника и могут быть отредактированы пользователем при необходимости (удалены, изменены, добавлены новые). Обратите внимание, что данный справочник является уже 5-ым на сервисе.

### 3. Результат:
- **ID**-  Из справочника, приложение 3. ID
- **КПГЗ** Выбранный пользователем или соответствующий выбранному шаблону ТЗ
- **СПГЗ** Один или несколько определенный сервисом. Може быть пустым, при этом не для каждой работы в смете должна (нужна) быть выявлена позиция СПГЗ
- **Объем** Подтягивается из сметы: Кол-во единиц
- **Единица измерения** Подтягивается из сметы: Единица измерения
- **Цена за единицу ТРУ, руб** *Рассчитываемое поле:* Стоимость за ТРУ, руб / Количество
- **Стоимость за ТРУ, руб** Подтягивается из сметы: ВСЕГО затрат, руб. При корректировке пол вручную должен быть произведен пересчет с предупреждением совпадения общей суммы по смете или возможность перераспределить остаток по всем работам в смете поровну или суммировать с выбранными позициями
- **Адрес** Вставляется при первоначальном выборе разбираемой сметы. При незаполненном поле осуществляется поиск адреса внутри сметы.

In [1]:
import os
import pandas as pd
import numpy as np
import re
from functools import reduce

pd.set_option('display.max_columns', 500)
pd.options.display.max_colwidth=250

In [2]:
# Загрузка сметы в гугл-документы 
url = 'https://docs.google.com/spreadsheets/d/1UWd16ahFAVwoc1CuJErjR5kZnbMNNp7c5FRBPedaQsQ/edit?usp=sharing'
url.split('/')
id = url.split('/')[5]
df = pd.read_excel(f'https://docs.google.com/spreadsheets/d/{id}/export?format=xlsx', sheet_name = 0, skiprows=9, header=None)

   
# Удаляем "шабку" внизу
df.drop(df.tail(8).index,inplace=True)

# Удалим строки с нан
df.dropna(axis=0, how='all', inplace=True)

# Удалим столбцы с нан
df.dropna(axis=1, how='all', inplace=True)

#display(df.head())
#df.tail()
df

,0,1,2,3,4,5,6,7,8,9,10,15,16,17,18,19,20,21,30,31
0,ЛОКАЛЬНАЯ СМЕТА №,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(локальный сметный расчет),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"ГБОУ Школа № 1788 г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1_(Рем)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"(наименование работ и затрат, наименование объекта)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Основание: чертежи №,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,"Итого по локальной смете: Благоустройство территории, прилегающей к ГБОУ Школа № 1788 по адресу: г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7376978.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Итого по локальной смете: Благоустройство территории, прилегающей к ГБОУ Школа № 1788 по адресу: г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
473,"Итого по смете: ГБОУ Школа № 1788 г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1_(Рем)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7376978.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Итого по смете: ГБОУ Школа № 1788 г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1_(Рем)"
474,NaN,NaN,Всего,NaN,NaN,NaN,NaN,NaN,7376978.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475,NaN,NaN,НДС 20%,NaN,NaN,NaN,NaN,NaN,1475395.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Уберем "шабку" из анализа

In [3]:
word = "адрес"
mylist = df[0].tolist()

try:
    df['address'] = next((s for s in mylist if word in s), None).split('адресу: ')[1]
except:
    df['address'] = ''
        

In [4]:
address = (
    df.loc[21:]
    .reset_index(drop=True)
)
try:
    address['address'] = list(address[0])[0].split('адресу: ')[1]
except:
    address['address'] = ''
address.head()

,0,1,2,3,4,5,6,7,8,9,10,15,16,17,18,19,20,21,30,31,address
0,"Локальная смета: Благоустройство территории, прилегающей к ГБОУ Школа № 1788 по адресу: г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Локальная смета: Благоустройство территории, прилегающей к ГБОУ Школа № 1788 по адресу: г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",NaN,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
1,"Раздел: Ремонт асфальтобетонного покрытия тротуаров (155,5 м2)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
2,1,2.1-3301-2-1/1,Исправление профиля щебеночных оснований с добавлением нового материала,1000 м2,0.04665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,513.91,513.91,73.42,73.42,2854.97,1761.92,NaN,NaN,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
3,NaN,NaN,"Объем: 0,04665=(1,555*0,3)/10",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
4,NaN,NaN,ЗП,NaN,NaN,15737.51,NaN,1,1,734.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"


In [5]:
# Выделяем нужные столбцы
df_new = address[[1,2,3,4,5,15, 'address']]
# Удалим строки с нан и переименуем столбцы
df_new = (
    df_new
    .dropna(axis=0, how='all')
    .reset_index(drop=True)
    .replace(np.nan, '')
    .rename(columns={1: 'code', 2: 'work', 3: 'count', 4: 'unit', 5: 'unit_price', 15: 'total'})
)
# 'code'- код\ Шифр расценки и коды ресурсов
# 'work' - наименование работы 
# 'count' - единица измерения 
# 'unit' - кол-во единиц 
# 'unit_price - цена на ед. изм. руб. 
# 'total' - ВСЕГО затрат, руб.

df_new

,code,work,count,unit,unit_price,total,address
0,,,,,,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
1,,,,,,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
2,2.1-3301-2-1/1,Исправление профиля щебеночных оснований с добавлением нового материала,1000 м2,0.04665,,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
3,,"Объем: 0,04665=(1,555*0,3)/10",,,,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
4,,ЗП,,,15737.51,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
...,...,...,...,...,...,...,...
414,,,,,,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
415,,,,,,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
416,,Всего,,,,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
417,,НДС 20%,,,,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"


In [6]:
data1 = df_new['work'].value_counts().reset_index()
data1 = data1[(data1['work'] < 6)]
data1

,index,work
9,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 20-40 мм",5
10,"Щебень из естественного камня для строительных работ, марка 600-400, фракция 5-10 мм",4
11,"Пигменты сухие для красок, кислотный желтый",4
12,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 10-20 мм",3
13,Перевозка строительного мусора автосамосвалами грузоподъемностью до 10 т на расстояние 1 км - при механизированной погрузке,3
...,...,...
85,"Объем: 7,651008=76,51008*0,1",1
86,"Объем: 76,51008=5,385*14,208",1
87,Замена бортового камня бетонного во дворовых территориях,1
88,"Объем: 58,671=65,19*0,9",1


In [7]:
list_work = list(data1['index'])
list_work.append('')
#list_work

In [8]:
dfisin = (
    df_new[df_new['work'].isin(list_work)]
    .reset_index(drop=True)
    #.drop(columns={'index'})
)

dfisin = dfisin[~(dfisin['code'] == "") | ~(dfisin['total'] == "")]

# Преобразуем типы
dfisin['total'] = dfisin['total'].replace('', np.nan).fillna(0).astype('float')
dfisin['unit'] = dfisin['unit'].replace('', np.nan).fillna(0).astype('float')

dfisin.info()
dfisin.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126 entries, 2 to 182
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   code        126 non-null    object 
 1   work        126 non-null    object 
 2   count       126 non-null    object 
 3   unit        126 non-null    float64
 4   unit_price  126 non-null    object 
 5   total       126 non-null    float64
 6   address     126 non-null    object 
dtypes: float64(2), object(5)
memory usage: 7.9+ KB


,code,work,count,unit,unit_price,total,address
2,2.1-3301-2-1/1,Исправление профиля щебеночных оснований с добавлением нового материала,1000 м2,0.046650,,0.00,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
4,21.1-12-35,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 10-20 мм",м3,-0.536475,1908.27,0.00,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
5,21.1-12-36,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 20-40 мм",м3,-2.565750,1806.27,0.00,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
6,21.1-12-29,"Щебень из естественного камня для строительных работ, марка 600-400, фракция 5-10 мм",м3,3.102225,1487.52,0.00,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
7,,,,0.000000,,11767.33,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"


In [9]:
df1 = (
    dfisin
    .drop(['total'], axis=1)
    #.query('code != ""')
    .reset_index(drop=True)
)
df1.info()
df1.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   code        126 non-null    object 
 1   work        126 non-null    object 
 2   count       126 non-null    object 
 3   unit        126 non-null    float64
 4   unit_price  126 non-null    object 
 5   address     126 non-null    object 
dtypes: float64(1), object(5)
memory usage: 6.0+ KB


,code,work,count,unit,unit_price,address
0,2.1-3301-2-1/1,Исправление профиля щебеночных оснований с добавлением нового материала,1000 м2,0.046650,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
1,21.1-12-35,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 10-20 мм",м3,-0.536475,1908.27,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
2,21.1-12-36,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 20-40 мм",м3,-2.565750,1806.27,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"


In [10]:
df2 = (
    dfisin[['total']]
    .reset_index(drop=True)
)
df2.info()
df2.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   total   126 non-null    float64
dtypes: float64(1)
memory usage: 1.1 KB


,total
0,0.0
1,0.0
2,0.0


In [11]:
# Добавим строки вниз
df3 = pd.DataFrame({'total':[0,0,0]})
df2 = pd.concat([df2, df3], ignore_index=True)
df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   total   129 non-null    float64
dtypes: float64(1)
memory usage: 1.1 KB


,total
0,0.00
1,0.00
2,0.00
3,0.00
4,11767.33


In [12]:
# Удалим первые строки
df2 = df2.loc[3:127].reset_index(drop=True)
df2.info()
df2.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   total   125 non-null    float64
dtypes: float64(1)
memory usage: 1.1 KB


,total
0,0.00
1,11767.33
2,0.00


In [13]:
df4 = (
    df1
    .join(df2, how='left')
    .query('code != ""')
    .reset_index(drop=True)
)
df4.info()
df4.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   code        80 non-null     object 
 1   work        80 non-null     object 
 2   count       80 non-null     object 
 3   unit        80 non-null     float64
 4   unit_price  80 non-null     object 
 5   address     80 non-null     object 
 6   total       80 non-null     float64
dtypes: float64(2), object(5)
memory usage: 4.5+ KB


,code,work,count,unit,unit_price,address,total
0,2.1-3301-2-1/1,Исправление профиля щебеночных оснований с добавлением нового материала,1000 м2,0.046650,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
1,21.1-12-35,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 10-20 мм",м3,-0.536475,1908.27,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",11767.33
2,21.1-12-36,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 20-40 мм",м3,-2.565750,1806.27,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
3,21.1-12-29,"Щебень из естественного камня для строительных работ, марка 600-400, фракция 5-10 мм",м3,3.102225,1487.52,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
4,2.1-3103-18-1/1,"Устройство покрытий из асфальтобетонных смесей вручную, толщина 4 см (5 см)",100 м2,1.555000,,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",59924.59


## Найдем ключевые работы

Можно найти по шифру или по названию. Загрузим единый справочник СН и ТСН

In [14]:
url = 'https://docs.google.com/spreadsheets/d/1S4GHtIwV5TdGPSQ92bWkMzMSVH8WzEVbMSTB8iF4NFA/edit?usp=sharing'
url.split('/')
id = url.split('/')[5]
sn_tsn = pd.read_excel(f'https://docs.google.com/spreadsheets/d/{id}/export?format=xlsx')
sn_tsn.head()

,code,work,count,total
0,1.1-3101-1-1/1,Разработка грунта внутри здания в котлованах,100 м3,243896.78
1,1.1-3101-1-2/1,Разработка грунта внутри здания в траншеях,100 м3,197319.19
2,1.1-3101-2-1/1,Подсыпка грунта при изменении уровня пола в здании,100 м3,165610.55
3,1.1-3101-2-2/1,Выемка грунта с погрузкой при изменении уровня пола в здании,100 м3,213235.79
4,1.1-3101-3-1/1,Разработка грунта экскаваторами в стесненных условиях,100 м2,75255.15


In [15]:
#  Проведем сопоставление данных в загруженной смете и внутреннем справочнике ТСН, СН.
df_sntsn = (
    df4[['code', 'work']]
    .merge(sn_tsn[['code', 'work', 'count']], how='left', on=['code', 'work'])
)

# Уберем пустые строки
df_sntsn = df_sntsn.loc[~(df_sntsn['code'] == '')].reset_index(drop=True)

df_sntsn

,code,work,count
0,2.1-3301-2-1/1,Исправление профиля щебеночных оснований с добавлением нового материала,1000 м21
1,21.1-12-35,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 10-20 мм",NaN
2,21.1-12-36,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 20-40 мм",NaN
3,21.1-12-29,"Щебень из естественного камня для строительных работ, марка 600-400, фракция 5-10 мм",NaN
4,2.1-3103-18-1/1,"Устройство покрытий из асфальтобетонных смесей вручную, толщина 4 см (5 см)",NaN
...,...,...,...
75,5.3-3103-11-1/1,Устройство наливного полиуретанового покрытия спортивных площадок и беговых дорожек толщиной 10 мм,100 м2
76,21.1-6-101,"Пигменты сухие для красок, кислотный желтый",NaN
77,5.3-3103-11-2/1,"Устройство наливного полиуретанового покрытия спортивных площадок и беговых дорожек, добавляется на 2 мм толщины покрытия",100 м2
78,21.1-6-101,"Пигменты сухие для красок, кислотный желтый",NaN


## 2. Разделы и работы, в которых употребляются словосочетания ниже, не отражающие суть ключевых работ являются исключениями и не должны участвовать в дальнейшем разборе сметы: 
- Вывоз мусора 
- Погрузка мусора 
- Перевозка мусора 
- Возвратные материалы 
- Пуско-наладочные работы и т.д. 

Данные исключения сформированы в сервисе в виде справочника и могут быть отредактированы пользователем при необходимости (удалены, изменены, добавлены новые). Обратите внимание, что данный справочник является уже 5-ым на сервисе.

### Нужно сделать новый 5й справочник с работами исключениями

In [16]:
# Вытащим новую таблицу исключений
words = ['мусор', 'возвратн', 'пуско-налад', 'свалк', 'отход']
list = '|'.join(words)
ignor_df = (
    df_sntsn
    .loc[df_sntsn['work'].str.contains(list, regex=True)]
    .sort_values(by='code')
    .reset_index(drop=True)
)
ignor_df.to_excel('ignor_df.xlsx', index=False)
display(ignor_df.shape)
display(ignor_df.sample(3))

# Список уникальных кодов для исключения
ignor_code = ignor_df['code'].unique().tolist()
#ignor_code

(18, 3)

,code,work,count
14,21.25-0-5,"Стоимость приемки отходов строительства и сноса (боя кирпичной кладки, бетонных и железобетонных изделий, отходов бетона и железобетона, асфальтобетона в кусковой форме) для переработки дробильными комплексами",NaN
10,2.12-3105-5-1/1,Погрузка вручную строительного мусора в самосвал,т1
17,9999990001,Масса мусора,NaN


In [17]:
# Вытащим вспомогательные работы
words = ['азборка', 'оски', 'выравниваю']
list = '|'.join(words)
secondary_work = (
    df_sntsn
    .loc[df_sntsn['work'].str.contains(list, regex=True)]
    .sort_values(by='code')
    .reset_index(drop=True)
)

display(secondary_work)

# Список уникальных кодов 
secondary_work_list = secondary_work['code'].unique().tolist()
#secondary_work_list

,code,work,count
0,1.10-3404-2-1/1,Разборка дощатых покрытий,100 м2
1,2.1-3204-6-1/1,Разборка бортовых камней на бетонном основании,100 м1
2,2.1-3303-1-1/1,Устройство подстилающих и выравнивающих слоев оснований из песка,100 м31
3,2.1-3303-1-2/1,Устройство подстилающих и выравнивающих слоев оснований из щебня,100 м31
4,2.1-3303-1-2/1,Устройство подстилающих и выравнивающих слоев оснований из щебня,100 м31
5,21.9-12-11,"Доски хвойных пород для покрытия пола, со шпунтом и гребнем, антисептированные, толщина 27 мм",NaN


In [18]:
# Работы где появилась единица измерения, соответствуют СПГЗ. Выявим их
df_sntsn = (
    df_sntsn
    .query('code != @ignor_code & code != @secondary_work_list & count != 0')
    .drop_duplicates(keep='first')
    .replace(np.nan, 0)
    .reset_index(drop=True)
)

df_sntsn

,code,work,count
0,2.1-3301-2-1/1,Исправление профиля щебеночных оснований с добавлением нового материала,1000 м21
1,21.1-12-35,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 10-20 мм",0
2,21.1-12-36,"Щебень из естественного камня для строительных работ, марка 1200-800, фракция 20-40 мм",0
3,21.1-12-29,"Щебень из естественного камня для строительных работ, марка 600-400, фракция 5-10 мм",0
4,2.1-3103-18-1/1,"Устройство покрытий из асфальтобетонных смесей вручную, толщина 4 см (5 см)",0
5,21.3-3-18,"Смеси асфальтобетонные дорожные горячие мелкозернистые, марка I, тип Б",0
6,21.3-3-34,"Смеси асфальтобетонные дорожные горячие песчаные, тип Д, марка III",0
7,1.10-3403-2-1/1,"Устройство покрытий дощатых толщиной, мм 28",100 м2
8,КА п. 1,"Доска террасная ДПК, размер 28х140х4000 мм.\n875,00 = [1 050 / 1,2]",0
9,21.1-11-46,Гвозди строительные,0


In [19]:
# Вытащим ключевые работы
kw_l = ['стройство', 'становка' 'покрыт', 'становка', 'кладка', 'краска']
lst = '|'.join(kw_l)
key_works = (
    df_sntsn
    .loc[df_sntsn['work'].str.contains(lst, regex=True)]
    .sort_values(by='code')
    .drop_duplicates(keep='first')
    .reset_index(drop=True)
    #.rename(columns={'work': 'spgz'})
)

display(key_works.head(20))

# Список уникальных кодов 
key_works_list = key_works['code'].unique().tolist()
#key_works_list

,code,work,count
0,1.10-3403-2-1/1,"Устройство покрытий дощатых толщиной, мм 28",100 м2
1,1.14-3203-14-7/1,Окраска масляными составами за два раза металлических поверхностей решеток и оград,100 м2
2,2.1-3103-17-1/1,"Устройство покрытий тротуаров из бетонной плитки типа ""Брусчатка"" рядовым или паркетным мощением",100 м21
3,2.1-3103-18-1/1,"Устройство покрытий из асфальтобетонных смесей вручную, толщина 4 см (5 см)",0
4,2.1-3203-1-5/2,Установка бортовых камней бетонных газонных и садовых при цементобетонных покрытиях,100 м1
5,5.3-3103-11-1/1,Устройство наливного полиуретанового покрытия спортивных площадок и беговых дорожек толщиной 10 мм,100 м2
6,5.3-3103-11-2/1,"Устройство наливного полиуретанового покрытия спортивных площадок и беговых дорожек, добавляется на 2 мм толщины покрытия",100 м2
7,5.4-3203-12-1/1,Сплошная укладка готового газона в рулонах на горизонтальных поверхностях или откосах с уклоном на круче 1:2,100 м2


### По этой смете получилось 8 ключевых работы.

### 3. Результат:
- **ID- id**-  Из справочника, приложение 3. ID
- **КПГЗ- kpgz** Выбранный пользователем или соответствующий выбранному шаблону ТЗ
- **СПГЗ- spgz** Один или несколько определенный сервисом. Може быть пустым, при этом не для каждой работы в смете должна (нужна) быть выявлена позиция СПГЗ
- **Объем- unit** Подтягивается из сметы: Кол-во единиц
- **Единица измерения- count** Подтягивается из сметы: Единица измерения
- **Цена за единицу ТРУ, руб- unit_price** *Рассчитываемое поле:* Стоимость за ТРУ, руб / Количество
- **Стоимость за ТРУ, руб- total** Подтягивается из сметы: ВСЕГО затрат, руб. При корректировке пол вручную должен быть произведен пересчет с предупреждением совпадения общей суммы по смете или возможность перераспределить остаток по всем работам в смете поровну или суммировать с выбранными позициями
- **Адрес** Вставляется при первоначальном выборе разбираемой сметы. При незаполненном поле осуществляется поиск адреса внутри сметы.

In [20]:
# Добавим столбцы со стоимостью
key_works = (
    key_works
    .merge(df4, how= 'left', on=['code', 'work'])
    .drop('count_x', axis=1)
    .rename(columns={'count_y': 'count'})
)
key_works['unit_price'] = round(key_works['total'] / key_works['unit'], 2)
key_works

,code,work,count,unit,unit_price,address,total
0,1.10-3403-2-1/1,"Устройство покрытий дощатых толщиной, мм 28",100 м2,2.800,0.00,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
1,1.14-3203-14-7/1,Окраска масляными составами за два раза металлических поверхностей решеток и оград,100 м2,6.600,0.00,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
2,2.1-3103-17-1/1,"Устройство покрытий тротуаров из бетонной плитки типа ""Брусчатка"" рядовым или паркетным мощением",100 м2,14.840,0.00,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
3,2.1-3103-18-1/1,"Устройство покрытий из асфальтобетонных смесей вручную, толщина 4 см (5 см)",100 м2,1.555,38536.71,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",59924.59
4,2.1-3103-18-1/1,"Устройство покрытий из асфальтобетонных смесей вручную, толщина 4 см (5 см)",100 м2,3.920,38536.71,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",151063.91
5,2.1-3103-18-1/1,"Устройство покрытий из асфальтобетонных смесей вручную, толщина 4 см (5 см)",100 м2,8.280,38536.71,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",319083.98
6,2.1-3203-1-5/2,Установка бортовых камней бетонных газонных и садовых при цементобетонных покрытиях,100 м,5.385,799.20,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",4303.69
7,5.3-3103-11-1/1,Устройство наливного полиуретанового покрытия спортивных площадок и беговых дорожек толщиной 10 мм,100 м2,3.920,0.00,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
8,5.3-3103-11-1/1,Устройство наливного полиуретанового покрытия спортивных площадок и беговых дорожек толщиной 10 мм,100 м2,8.280,0.00,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",0.00
9,5.3-3103-11-2/1,"Устройство наливного полиуретанового покрытия спортивных площадок и беговых дорожек, добавляется на 2 мм толщины покрытия",100 м2,3.920,69403.34,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1",272061.09


### Откроем ключевые СПГЗ и КПГЗ с кодами-шифрами

In [21]:
url = 'https://docs.google.com/spreadsheets/d/1a4n2yJjFjJQWJ26G5x-Ict43ALCLrP1S1UcUqAvFLHY/edit?usp=sharing'
url.split('/')
id = url.split('/')[5]
spgz = pd.read_excel(f'https://docs.google.com/spreadsheets/d/{id}/export?format=xlsx')
spgz = (
    spgz
    .drop(columns={'Unnamed: 3'})
    .rename(columns={'Шифр расценки и коды ресурсов': 'code',
                    'Наименование работ и затрат': 'work',
                    'Ед. изм.': 'count',
                    'Наименование СПГЗ': 'spgz',
                    'ID': 'id',
                    'КПГЗ': 'kpgz',
                    'Единицы измерения': 'unit'
    })
)
spgz.head(3)

,code,work,count,spgz,id,kpgz,unit
0,5.3-3101-1-1/1,Ремонт садовых дорожек из щебня добавлением слоя 5 см,м2,Ремонт дорожно-тропиночной сети с покрытием из щебня на особо охраняемых природных территориях,10548507.0,02.03.05 СТРОИТЕЛЬСТВО НА ОСОБО ОХРАНЯЕМЫХ ПРИРОДНЫХ ТЕРРИТОРИЯХ (ООПТ),Квадратный метр
1,5.3-3101-1-2/1,"Ремонт садовых дорожек из щебня, добавлять или удалять на каждый 1 см к поз. 3-3101-1-1",м2,сопутсвующая работа к 5.3-3101-1-1/1,NaN,NaN,NaN
2,5.3-3101-3-1/1,"Ремонт грунтовых дорожек при площади выбоин до 0,5 м2 и глубине 5-10 см",100 м2,Ремонт дорожно-тропиночной сети с грунтовым покрытием при благоустройстве парков,77612547.0,"02.03.04 СТРОИТЕЛЬСТВО ПАРКОВ, МЕСТ ОТДЫХА И ДОСУГА",Квадратный метр


In [22]:
# Есть ли дубликаты
spgz.duplicated().sum()

18

In [23]:
print("Размер таблицы до удаления дубликатов", spgz.shape)
spgz = spgz.drop_duplicates(keep='first')
print("После удаления дубликатов", spgz.shape)

Размер таблицы до удаления дубликатов (3929, 7)
После удаления дубликатов (3911, 7)


In [24]:
# Объединим таблицы
estimate_spgz = key_works[['code', 'work', 'unit_price', 'total']].merge(spgz, how='left', on=['code', 'work'])
print("Дубликатов в новой таблице", estimate_spgz.duplicated().sum())
print("Размер таблицы", estimate_spgz.shape)
estimate_spgz.sample(3)

Дубликатов в новой таблице 14
Размер таблицы (44, 9)


,code,work,unit_price,total,count,spgz,id,kpgz,unit
7,5.3-3103-11-1/1,Устройство наливного полиуретанового покрытия спортивных площадок и беговых дорожек толщиной 10 мм,0.00,0.0,100 м2,Замена покрытия акрилового (хард) в рамках благоустройства территории,91742866.0,02.03.03.11 ОБУСТРОЙСТВО ПОКРЫТИЙ И ЭЛЕМЕНТОВ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Квадратный метр
33,5.3-3103-11-1/1,Устройство наливного полиуретанового покрытия спортивных площадок и беговых дорожек толщиной 10 мм,0.00,0.0,100 м2,Устройство резинового покрытия детской игровой площадки при благоустройстве парков,NaN,NaN,NaN
37,5.4-3203-12-1/1,Сплошная укладка готового газона в рулонах на горизонтальных поверхностях или откосах с уклоном на круче 1:2,10226.99,102269.9,100 м2,Устройство газона в рамках благоустройства территории,91740543.0,02.03.03.03 ОЗЕЛЕНЕНИЕ ТЕРРИТОРИЙ,Квадратный метр


In [25]:
# Выделим столбцы для итоговой таблицы. Удалим дубликаты, если есть.
final = (
    estimate_spgz[['id', 'kpgz', 'spgz', 'count', 'unit', 'unit_price', 'total']]
    .merge(key_works[['count', 'address']], how='left', on='count')
    .drop_duplicates(keep='first')
    .replace(np.nan, 0)
    .query('id != 0')
    .reset_index(drop=True)
)
print("Размер финальной таблицы", final.shape)
final

Размер финальной таблицы (18, 8)


,id,kpgz,spgz,count,unit,unit_price,total,address
0,91742866.0,02.03.03.11 ОБУСТРОЙСТВО ПОКРЫТИЙ И ЭЛЕМЕНТОВ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Замена покрытия акрилового (хард) в рамках благоустройства территории,100 м2,Квадратный метр,0.00,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
1,91739067.0,02.06.05.04 РАБОТЫ РЕМОНТНО-ВОССТАНОВИТЕЛЬНЫЕ СВЯЗАННЫЕ С ПОКРЫТИЯМИ И ЭЛЕМЕНТАМИ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Замена покрытия акрилового (хард) в рамках ремонтно-восстановительных работ на объектах благоустройства,100 м2,Квадратный метр,0.00,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
2,91738688.0,02.03.03.11 ОБУСТРОЙСТВО ПОКРЫТИЙ И ЭЛЕМЕНТОВ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Замена покрытия из резиновой крошки в рамках благоустройства территории,100 м2,Квадратный метр,0.00,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
3,60046142.0,02.03.12 РАБОТЫ ПО БЛАГОУСТРОЙСТВУ ТЕРРИТОРИЙ ОБЪЕКТОВ СПОРТА,Замена покрытия из резиновой крошки в рамках благоустройства территорий объектов спорта,100 м2,Квадратный метр,0.00,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
4,46323952.0,"02.03.04 СТРОИТЕЛЬСТВО ПАРКОВ, МЕСТ ОТДЫХА И ДОСУГА",Устройство EPDM покрытия детской игровой площадки при благоустройстве парков,100 м2,Квадратный метр,0.00,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
5,46323931.0,"02.03.04 СТРОИТЕЛЬСТВО ПАРКОВ, МЕСТ ОТДЫХА И ДОСУГА",Устройство EPDM покрытия спортивной площадки при благоустройстве парков,100 м2,Квадратный метр,0.00,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
6,91741271.0,02.03.03.11 ОБУСТРОЙСТВО ПОКРЫТИЙ И ЭЛЕМЕНТОВ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Устройство покрытия акрилового (хард) в рамках благоустройства территории,100 м2,Квадратный метр,0.00,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
7,91742563.0,02.06.05.04 РАБОТЫ РЕМОНТНО-ВОССТАНОВИТЕЛЬНЫЕ СВЯЗАННЫЕ С ПОКРЫТИЯМИ И ЭЛЕМЕНТАМИ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Устройство покрытия акрилового (хард) в рамках ремонтно-восстановительных работ на объектах благоустройства,100 м2,Квадратный метр,0.00,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
8,91739664.0,02.03.03.11 ОБУСТРОЙСТВО ПОКРЫТИЙ И ЭЛЕМЕНТОВ СОПРЯЖЕНИЯ ТЕРРИТОРИЙ,Устройство покрытия из резиновой крошки в рамках благоустройства территории,100 м2,Квадратный метр,0.00,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"
9,60046236.0,02.03.12 РАБОТЫ ПО БЛАГОУСТРОЙСТВУ ТЕРРИТОРИЙ ОБЪЕКТОВ СПОРТА,Устройство покрытия из резиновой крошки в рамках благоустройства территорий объектов спорта,100 м2,Квадратный метр,0.00,0.0,"г. Москва, пос. Внуковское, ул. Летчика Грицевца, д. 5, к. 1"


In [26]:
# Сохраним результат
final.to_excel('estimate_final.xlsx', index=False)